In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [112]:
train = pd.read_csv("data/bicikelj_train.csv")
train["timestamp"] = pd.to_datetime(train["timestamp"])

test = pd.read_csv("data/bicikelj_test.csv")
test["timestamp"] = pd.to_datetime(test["timestamp"])

In [150]:
train = pd.read_csv("data/train_processed.csv")
test = pd.read_csv("data/test_processed.csv")

In [114]:
# Extract new features
def get_time_based_features(timestamp):
    # From datetime64 get month, day, hour, dayOfWeek and isHoliday
    timestamp = np.datetime64(timestamp)
    dt_python = timestamp.astype(datetime.datetime)

    # Extract month, day, hour, and day_of_week
    month = dt_python.month
    day = dt_python.day
    hour = dt_python.hour
    minute = dt_python.minute
    day_of_week = dt_python.weekday()

    # Check for holidays
    is_holiday = 0
    if dt_python.month == 8 and dt_python.day in [15, 17]:
        is_holiday = 1
    elif dt_python.month == 9 and dt_python.day in [15, 23]:
        is_holiday = 1
        
        
    is_weekend = 0
    if dt_python.weekday() in [5, 6]:
        is_weekend = 1
   
    is_night = 0
    if hour < 6 or hour > 22:
        is_night = 1
        
    school_holiday = 0
    if 6 <= month <= 8:
        school_holiday = 1
   
    return month, day, hour, minute, day_of_week, is_holiday, is_weekend, is_night, school_holiday


In [151]:
# Get list of stations
stations = []
for i in range(1, 84):
    stations.append(train.columns[i])

In [153]:
to_drop = []
for station in stations:
    to_drop.append(f"{station}_150min")
train = train.drop(to_drop, axis=1)
test = test.drop(to_drop, axis=1)

In [116]:
def find_closest_smaller_row(df, timestamp, hour_offset, thresh=15):
    print(timestamp)
    target_timestamp = pd.to_datetime(timestamp) - pd.DateOffset(hours=hour_offset)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    smaller_rows = df[(pd.to_datetime(df["timestamp"]) <= target_timestamp) & (df["timestamp"] > target_timestamp - datetime.timedelta(minutes=thresh))]
    if smaller_rows.empty:
        # Exclude from training
        return None
    closest_index = np.argmin(np.abs(target_timestamp - pd.to_datetime(smaller_rows["timestamp"])))
    print(smaller_rows["timestamp"], closest_index)
    closest_row = smaller_rows.iloc[closest_index]
    
    return closest_row

In [117]:
def closest(df, timestamp, minute_offset=60, thresh=15):
    target_ts = pd.to_datetime(timestamp) - pd.DateOffset(minutes=minute_offset) 
    data = df.copy()
    data["timestamp"] = pd.to_datetime(data["timestamp"])
    data = data[(data["timestamp"] <= target_ts + pd.DateOffset(minutes=5)) & (data["timestamp"] > target_ts - pd.DateOffset(minutes=thresh))]
    if len(data) == 0: 
        return None
    
    # print(target_ts, "\n", data["timestamp"])
    closest_index = np.argmin(np.abs(target_ts - pd.to_datetime(data["timestamp"])))
    # print("Closest", data.iloc[closest_index]["timestamp"])
    return data.iloc[closest_index]

In [118]:
closest(train, "2022-08-02 14:10:00")

In [119]:
train.shape

(6906, 377)

In [276]:
# for station in stations:
#     train[f"{station}_1hr"] = [None for _ in range(train.shape[0])]
#     train[f"{station}_2hr"] = [None for _ in range(train.shape[0])]
    
# train.columns

In [137]:
 # offset_df = pd.DataFrame(columns=[f"{station}_1hr" for station in stations] + [f"{station}_2hr" for station in stations])

for i, timestamp in enumerate(test["timestamp"]):
#     # row_1hr = closest(train, timestamp, 1)
#     # row_2hr = closest(train, timestamp, 2)
    row_150 = closest(train, timestamp, 150)
    
        
    if row_150 is not None:
        # offset_df.append(row_1hr[stations], ignore_index=True)
        test.loc[i, [f"{station}_150min" for station in stations]] = row_150[stations].values
    
#     # if row_1hr is not None:
#     #     # offset_df.append(row_1hr[stations], ignore_index=True)
#     #     train.loc[i, [f"{station}_1hr" for station in stations]] = row_1hr[stations].values
        
# #     if row_2hr is not None:
# #         train.loc[i, [f"{station}_2hr" for station in stations]] = row_2hr[stations].values
    
    
    if i % 10 == 0:
        print(f"Processed {i} rows")
    
# # Remove all rows with None
# train = train.dropna()
#     train[f"{station}_1hr"] = train.columns.apply(lambda station: find_closest_smaller_row(train, time, 1, station))
#     train[f"{station}_2hr"] = train["timestamp"].apply(lambda time: find_closest_smaller_row(train, time, 2, station))

Processed 0 rows
Processed 10 rows
Processed 20 rows
Processed 30 rows


In [138]:
test.columns

Index(['timestamp', 'PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE',
       'POGAČARJEV TRG-TRŽNICA', 'KONGRESNI TRG-ŠUBIČEVA ULICA',
       'CANKARJEVA UL.-NAMA', 'BREG', 'GRUDNOVO NABREŽJE-KARLOVŠKA C.',
       'MIKLOŠIČEV PARK', 'BAVARSKI DVOR', 'TRG OF-KOLODVORSKA UL.',
       ...
       'POVŠETOVA - KAJUHOVA_150min', 'SOSESKA NOVO BRDO_150min',
       'TEHNOLOŠKI PARK_150min', 'VOJKOVA - GASILSKA BRIGADA_150min',
       'GERBIČEVA - ŠPORTNI PARK SVOBODA_150min',
       'DOLENJSKA C. - STRELIŠČE_150min', 'ROŠKA - STRELIŠKA_150min',
       'LEK - VEROVŠKOVA_150min', 'VOKA - SLOVENČEVA_150min',
       'SUPERNOVA LJUBLJANA - RUDNIK_150min'],
      dtype='object', length=460)

In [15]:
# for i, timestamp in enumerate(test["timestamp"]):
#     row_1hr = closest(train, timestamp, 90)
# #     row_2hr = closest(train, timestamp, 2)
    
#     if row_1hr is not None:
# #         # offset_df.append(row_1hr[stations], ignore_index=True)
#         test.loc[i, [f"{station}_90min" for station in stations]] = row_1hr[stations].values
        
#     if row_2hr is not None:
#         test.loc[i, [f"{station}_2hr" for station in stations]] = row_2hr[stations].values
    
    
#     if i % 100 == 0:
#         print(f"Processed {i} rows")

In [279]:
# test.to_csv("data/enriched_test.csv", index=False)

In [280]:
# train = pd.read_csv("data/test_train.csv")
# test = pd.read_csv("data/enriched_test.csv")
# train = train.dropna()

In [281]:
# train.to_csv("data/test_train.csv", index=False)

In [14]:
d = train
# d = pd.read_csv("data/test_train.csv")
ix = 12
for station in stations:
    if d.iloc[ix][station] - d.iloc[ix + 18][f'{station}_90min'] != 0:
        print(f"{station} time: {d.iloc[ix]['timestamp']} - {d.iloc[ix + 12]['timestamp']}: n-bikes vs n_bikes_90min {d.iloc[ix][station]} {d.iloc[ix + 12][f'{station}_90min']}")

In [132]:
test.columns

Index(['timestamp', 'PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE',
       'POGAČARJEV TRG-TRŽNICA', 'KONGRESNI TRG-ŠUBIČEVA ULICA',
       'CANKARJEVA UL.-NAMA', 'BREG', 'GRUDNOVO NABREŽJE-KARLOVŠKA C.',
       'MIKLOŠIČEV PARK', 'BAVARSKI DVOR', 'TRG OF-KOLODVORSKA UL.',
       ...
       'SOSESKA NOVO BRDO_90min', 'TEHNOLOŠKI PARK_90min',
       'VOJKOVA - GASILSKA BRIGADA_90min',
       'GERBIČEVA - ŠPORTNI PARK SVOBODA_90min',
       'DOLENJSKA C. - STRELIŠČE_90min', 'ROŠKA - STRELIŠKA_90min',
       'LEK - VEROVŠKOVA_90min', 'VOKA - SLOVENČEVA_90min',
       'SUPERNOVA LJUBLJANA - RUDNIK_90min', 'feelslike'],
      dtype='object', length=377)

In [5]:
# Add new features to train
# train['month'], train['day'], train['hour'], train['minute'], train['day_of_week'], train['is_holiday'], train['is_weekend'], train['is_night'], train['school_holiday'] = zip(*train['timestamp'].map(get_time_based_features))
# test['month'], test['day'], test['hour'], test['minute'], test['day_of_week'], test['is_holiday'], test['is_weekend'], test['is_night'], test['school_holiday'] = zip(*test['timestamp'].map(get_time_based_features))


In [49]:
stations_meta = pd.read_csv("data/bicikelj_metadata.csv", sep="\t")
stations_meta.head()

,postaja,geo-visina,geo-sirina,total_space
0,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE,46.051367,14.506542,20
1,POGAČARJEV TRG-TRŽNICA,46.051093,14.507186,18
2,KONGRESNI TRG-ŠUBIČEVA ULICA,46.050388,14.504623,20
3,CANKARJEVA UL.-NAMA,46.052431,14.503257,26
4,BREG,46.046498,14.505148,20


In [286]:
# One hot encode days of week and hours
days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]

# for i, day in enumerate(days_of_week):
#     train[day] = (train["day_of_week"] == i).astype(int) 
#     test[day] = (test["day_of_week"] == i).astype(int)


# hours = [i for i in range(24)]
# for i in hours:
#     train[f"hour_{i}"] = (train["hour"] == i).astype(int)
#     test[f"hour_{i}"] = (test["hour"] == i).astype(int)

In [154]:
train.to_csv("data/train_processed.csv", index=False)
test.to_csv("data/test_processed.csv", index=False)

train = pd.read_csv("data/train_processed.csv")
test = pd.read_csv("data/test_processed.csv")

In [129]:
# Get list of stations
stations = []
for i in range(1, 84):
    stations.append(train.columns[i])

In [140]:
train_station_dataframes = {}
test_station_dataframes = {}

for station in stations:
            
    for n in range(2):
        if n == 0:
            dataframe = train
        else:
            dataframe = test
        station_df = pd.DataFrame({
            "timestamp": dataframe["timestamp"].values,
            "n_bikes": dataframe[station].values,
            "month": dataframe["month"].values,
            "day": dataframe["day"].values,
            "hour": dataframe["hour"].values,
            "minute": dataframe["minute"].values,
            "day_of_week": dataframe["day_of_week"].values,
            "is_holiday": dataframe["is_holiday"].values,
            "is_weekend": dataframe["is_weekend"].values,
            "is_night": dataframe["is_night"].values,
            "school_holiday": dataframe["school_holiday"].values,
            "n_bikes_1hr": dataframe[f"{station}_1hr"].values,
            "n_bikes_2hr": dataframe[f"{station}_2hr"].values,
            "n_bikes_90min": dataframe[f"{station}_90min"].values,
            "rain": dataframe["rain"].values,
            "rain_condition": dataframe["rain_condition"].values,
            "feelslike": dataframe["feelslike"].values,
            "temp": dataframe["temp"].values
        })
        
        days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]

        for i, day in enumerate(days_of_week):
            station_df[day] = dataframe[day].values


        hours = [i for i in range(24)]
        for i in hours:
            station_df[f"hour_{i}"] = dataframe[f"hour_{i}"].values   
            
        if n == 0:    
            train_station_dataframes[station] = station_df
        else: test_station_dataframes[station] = station_df
        
        # station = station.replace("/", "_")
        # if n == 0:
        #     station_df.to_csv(f"data/stations/{station}_train.csv", index=False)
        # else:
        #     station_df.to_csv(f"data/stations/{station}_test.csv", index=False)    
    # station_df.to_csv(f"data/stations/{station}.csv", index=False)

In [141]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [142]:
def linear_regression(X_train, X_test, y_train, y_test=None):
    # Fit model
   # lr = linear_model.Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, solver="auto", tol=1e-10) 
    lr = linear_model.Ridge() 
    lr.fit(X_train, y_train)
    
    # Predict and score
    y_pred = lr.predict(X_test)
    
    if y_test is not None:
        mse = mean_squared_error(y_test, y_pred)
        print(f"MSE: {mse}")
    
    return lr, y_pred

In [143]:
def random_forest(X_train, X_test, y_train, y_test=None):
    rf = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=0)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    return rf, y_pred

In [144]:
def kernel_ridge(X_train, X_test, y_train, y_test=None):
    # Fit model
    kr = KernelRidge(alpha=1, kernel="rbf") 
    # lr = linear_model.LinearRegression()
    kr.fit(X_train, y_train)
    
    # Predict and score
    y_pred = kr.predict(X_test)
    
    return kr, y_pred

In [145]:
def svr(X_train, X_test, y_train, y_test=None, kernel="rbf"):
    # Fit model
    svr = SVR() 
    # lr = linear_model.LinearRegression()
    svr.fit(X_train, y_train)
    
    # Predict and score
    y_pred = svr.predict(X_test)
    
    return svr, y_pred

##### Use train set to get good results first and then use test set on classroom

In [146]:
avg_mse = 0
# Perform linear regression on all stations
for station in stations:
    station_df = train_station_dataframes[station]
    
    X = station_df.drop(["timestamp", "n_bikes", "minute", "day_of_week"], axis=1)
    y = station_df["n_bikes"]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)   
        
    X_train_1hr = X_train[X_train.index % 2 == 0]
    X_train_2hr = X_train[X_train.index % 2 == 1]
    
    y_train_1hr = y_train[y_train.index % 2 == 0]
    y_train_2hr = y_train[y_train.index % 2 == 1]
    
    X_test_1hr = X_test[X_test.index % 2 == 0]
    X_test_2hr = X_test[X_test.index % 2 == 1]
    
    # No data for 1hr back
    X_train_2hr = np.array(X_train_2hr["n_bikes_2hr"]).reshape(-1, 1) 
    X_test_2hr = np.array(X_test_2hr["n_bikes_2hr"]).reshape(-1, 1)
    
    lr_1h, y_pred_1h = linear_regression(X_train_1hr, X_test_1hr, y_train_1hr)
    lr_2h, y_pred_2h = linear_regression(X_train_2hr, X_test_2hr, y_train_2hr)
    
    # rf_1h, y_pred_1h = random_forest(X_train_1hr, X_test_1hr, y_train_1hr)
    # rf_2h, y_pred_2h = random_forest(X_train_2hr, X_test_2hr, y_train_2hr)
    
    y_pred = np.empty(len(y_pred_1h) + len(y_pred_2h))
    y_pred[::2] = y_pred_1h
    # print(y_pred_1h, y_pred_2h)
    y_pred[1::2] = y_pred_2h
    
    avg_mse += mean_squared_error(y_test, y_pred)
    
    y_pred = pd.DataFrame({"n_bikes": y_pred})

avg_mse /= len(stations)
        
print(f"Score: {avg_mse}")

Score: 7.91629484791617


In [147]:
train.columns

Index(['timestamp', 'PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE',
       'POGAČARJEV TRG-TRŽNICA', 'KONGRESNI TRG-ŠUBIČEVA ULICA',
       'CANKARJEVA UL.-NAMA', 'BREG', 'GRUDNOVO NABREŽJE-KARLOVŠKA C.',
       'MIKLOŠIČEV PARK', 'BAVARSKI DVOR', 'TRG OF-KOLODVORSKA UL.',
       ...
       'POVŠETOVA - KAJUHOVA_150min', 'SOSESKA NOVO BRDO_150min',
       'TEHNOLOŠKI PARK_150min', 'VOJKOVA - GASILSKA BRIGADA_150min',
       'GERBIČEVA - ŠPORTNI PARK SVOBODA_150min',
       'DOLENJSKA C. - STRELIŠČE_150min', 'ROŠKA - STRELIŠKA_150min',
       'LEK - VEROVŠKOVA_150min', 'VOKA - SLOVENČEVA_150min',
       'SUPERNOVA LJUBLJANA - RUDNIK_150min'],
      dtype='object', length=460)

In [148]:
stations_meta.loc[stations_meta["postaja"] == stations[0]]["total_space"]

0    20
Name: total_space, dtype: int64

In [149]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2, include_bias=False)

# test = pd.read_csv("data/test_expanded.csv")
output = pd.read_csv("data/bicikelj_test.csv").copy()

# Perform linear regression on all stations
for station in stations:

    station_df = train_station_dataframes[station]
    
    X_train = station_df.drop(["timestamp", "n_bikes"], axis=1)
    y_train = station_df["n_bikes"]


    
    X_test = test_station_dataframes[station].drop(["timestamp", "n_bikes"], axis=1)
    
    X_train_1hr = X_train[X_train.index % 2 == 0]
    X_train_2hr = X_train[X_train.index % 2 == 1]
    y_train_1hr = y_train[y_train.index % 2 == 0]
    y_train_2hr = y_train[y_train.index % 2 == 1]
    X_test_1hr = X_test[X_test.index % 2 == 0]
    X_test_2hr = X_test[X_test.index % 2 == 1]
    
    # No data for 1hr back
    X_train_2hr = np.array(X_train_2hr.drop(["n_bikes_1hr", "n_bikes_90min"], axis=1))
    X_test_2hr = np.array(X_test_2hr.drop(["n_bikes_1hr", "n_bikes_90min"], axis=1))
    
    # X_train_1hr = poly.fit_transform(X_train_1hr)
    # X_test_1hr = poly.fit_transform(X_test_1hr)
    
    # X_train_2hr = poly.fit_transform(X_train_2hr)
    # X_test_2hr = poly.fit_transform(X_test_2hr)
    
    
    lr_1h, y_pred_1h = linear_regression(X_train_1hr, X_test_1hr, y_train_1hr)
    lr_2h, y_pred_2h = linear_regression(X_train_2hr, X_test_2hr, y_train_2hr)
    
    # rf_1h, y_pred_1h = random_forest(X_train_1hr, X_test_1hr, y_train_1hr)
    # rf_2h, y_pred_2h = random_forest(X_train_2hr, X_test_2hr, y_train_2hr)
    
    y_pred = np.empty(len(y_pred_1h) + len(y_pred_2h))
    y_pred[::2] = y_pred_1h
    # print(y_pred_1h, y_pred_2h)
    y_pred[1::2] = y_pred_2h
    
    y_pred = pd.DataFrame({"n_bikes": y_pred})
    
    # Set values in y_pred to 0 if negative
    y_pred[y_pred < 0] = 0
    
    total_space = stations_meta.loc[stations_meta["postaja"] == station]["total_space"].values[0]
    y_pred[y_pred > total_space] = total_space
    
    
    # m = lr_1h.coef_.argsort()[::-1]
    
    # for i in range(len(m)):
        
    #     print(X_train.columns[m[i]], lr.coef_[m[i]])
    output[station] = y_pred
    
output.to_csv("data/output.csv", index=False)

# n_bikes_3hr??